In [1]:
import pandas as pd
dataset_spotify = pd.read_csv('data.csv', header=0)
dataset_weather_alexandria = pd.read_csv('alexandria.csv', header=0)

In [2]:
merged_dataset = dataset_spotify.merge(dataset_weather_alexandria, on='date',)
merged_dataset.to_csv('./cleaned_data/merged.csv', index=False)

In [3]:
def binary_precipitation(value):
    if value != 0:
        return 1
    else:
        return 0


Convert all raining values into a binary, "Is raining" or not

In [4]:
merged_dataset['precipcover'] = merged_dataset['precipcover'].apply(binary_precipitation)


In [17]:
x = merged_dataset.drop(columns=['track_id','playlist_id','date','id','compound','name','tempmax','tempmin','temp','feelslikemax','feelslikemin','feelslike','dew','humidity','precip','precipprob','precipcover','preciptype','snow','snowdepth','windgust','windspeed','winddir','sealevelpressure','cloudcover','visibility','solarradiation','solarenergy','uvindex','severerisk','sunrise','sunset','moonphase','conditions','description','icon','stations'])
y = merged_dataset['precipcover']
x. fillna(0,inplace=True)

x.to_csv('./cleaned_data/cleaned.csv', index=False)
x.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,neg,neu,pos
0,0.418,0.794,11,-4.973,1,0.0816,0.080,0.000000,0.3290,0.178,129.434,241846,4,0.199,0.584,0.216
1,0.418,0.794,11,-4.973,1,0.0816,0.080,0.000000,0.3290,0.178,129.434,241846,4,0.199,0.584,0.216
2,0.658,0.294,4,-8.533,0,0.0321,0.422,0.000052,0.0749,0.358,74.038,278180,4,0.038,0.836,0.125
3,0.658,0.294,4,-8.533,0,0.0321,0.422,0.000052,0.0749,0.358,74.038,278180,4,0.038,0.836,0.125
4,0.424,0.606,6,-6.062,0,0.1590,0.106,0.000272,0.1440,0.397,213.788,342244,4,0.000,0.000,0.000


In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

Using a decision tree classifier, I am able to predict if it is raining based on the type of music I am listening to.

In [20]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree = tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

In [21]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5725190839694656


In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [23]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

In [24]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6335877862595419
